In [3]:
import gmsh
import numpy as np

In [46]:
gmsh.initialize()

gmsh.model.add("Room")
'''
Создание элементов комнаты
'''
# Параметры комнаты
room_width = 3.8
room_height = 3.1
room_length = 5.6

# Параметры окна
window_width = 1.6
window_height = 1.5
window_offset = 1.4   # отступ окна от пола

# Параметры батареи
heater_width = 1.5
heater_height = 1
heater_depth = 0.14
heater_offset = 0.2   # расстояние от пола до нижней части батареи

room = gmsh.model.occ.addBox(0, 0, 0, room_width, room_length, room_height)

x_window = (room_width - window_width) / 2
y_window = room_length
z_window = window_offset
window_surface = gmsh.model.occ.addRectangle(x_window, y_window, z_window, window_width, window_height)
gmsh.model.occ.rotate([(2,window_surface)], x_window, room_length, window_offset, 1, 0, 0, 90 * (np.pi / 180))

x_heater = x_window + (window_width - heater_width) / 2
y_heater = y_window - heater_depth
z_heater = heater_offset
heater = gmsh.model.occ.addBox(x_heater, y_heater, z_heater, heater_width, heater_depth, heater_height)
gmsh.model.occ.synchronize()

'''
    Создание физических групп для задания граничных условий в решателе
'''
gmsh.model.addPhysicalGroup(2, [window_surface], tag=2)
gmsh.model.setPhysicalName(2, 2, "Window Surface")

gmsh.model.addPhysicalGroup(3, [heater], tag=3)
gmsh.model.setPhysicalName(3, 3, "Heater")

cut_result = gmsh.model.occ.cut([(3, room)], [(3, heater)], removeTool=False)
gmsh.model.occ.synchronize() #обязательно делать эту операцию после вырезания, иначе не перестраивает сетку, может случится перекрытие сетки комнаты и батареи

air_volume = cut_result[0][0][1]
gmsh.model.addPhysicalGroup(3, [air_volume], tag=1)
gmsh.model.setPhysicalName(3, 1, "Air Volume")
gmsh.model.occ.synchronize()

'''
    Изменение размера сетки на локальных объектах.
    :param Mesh.CharacteristicLengthMin - должно коррелировать с размером сетки батареи
'''
gmsh.option.setNumber("Mesh.CharacteristicLengthMin", 0.1)
gmsh.option.setNumber("Mesh.CharacteristicLengthMax", 0.5)
gmsh.model.occ.synchronize()
heater_nodes = gmsh.model.getBoundary([(3, heater)], oriented=False, recursive=True)
gmsh.model.mesh.setSize(heater_nodes, 0.1)

gmsh.model.occ.synchronize()

gmsh.model.mesh.generate(3)
gmsh.write("room_with_refined_mesh_on_heater.msh2")
gmsh.write("room_with_refined_mesh_on_heater.vtk")


gmsh.finalize()